<a href="https://colab.research.google.com/github/danielx1611/ComputerVisionAssignments/blob/main/PinholeCameraModel/CameraCalibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Camera Calibration with Gradio UI

In [120]:
%pip install gradio numpy opencv-python matplotlib pytransform3d

In [121]:
from functools import partial
import time
import gradio
import os
import json
import numpy as np
import cv2 as cv
import glob
import matplotlib.pyplot as plt
import pytransform3d.camera as pc
import pytransform3d.transformations as pt
import io
from pathlib import Path
from PIL import Image

def cv2_imshow(img):
    cv.imshow("img", img)
    time.sleep(1)
    cv.destroyAllWindows()

CONTENT_DIR = Path(".")

try:
    import google.colab.patches as patches
    print("in collab")
    cv2_imshow = patches.cv2_imshow
    CONTENT_DIR = Path("/content")

    if not Path("ComputerVisionAssignments").exists():
      !git clone https://github.com/danielx1611/ComputerVisionAssignments

    # Move test files to the Colab local directory (i.e., /content/)
    if not Path("images").exists():
      !mkdir images
    !cp -r ComputerVisionAssignments/PinholeCameraModel/chess_jpg/* ./images
except:
    pass

in collab


### Camera Calibration

In [122]:
image_dir = CONTENT_DIR / "images"
output_dir = CONTENT_DIR / "output"
output_dir.mkdir(exist_ok=True)
images = list(image_dir.glob("*.jpeg")) + list(image_dir.glob("*.jpg"))


class Calibration:
    @staticmethod
    def run():
        # termination criteria
        criteria = (
            cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER,
            30,
            0.001,
        )  # 30: max_iter , 0.001 : accuracy

        # prepare object points, like (0,0,0), (1,0,0), ..., (6,5,0)
        objp = np.zeros((9 * 6, 3), np.float32)  #
        objp[:, :2] = np.mgrid[0:9, 0:6].T.reshape(
            -1, 2
        )  # reshaped to [42,2] only take X and Y


        calibration_data = {}

        CHECKERBOARD_SIZE = (9, 6)

        for fname in images:
            # Arrays to store object points and image points from all the images.
            objpoints: list[cv.typing.MatLike] = []  # 3d point in real world space [X,Y,Z]
            imgpoints: list[cv.typing.MatLike] = []  # 2d points in image plane. [X,Y]

            print(fname)
            img = cv.imread(str(fname))

            if img is None:
                print("Invalid image")
                continue

            gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

            # Find the chess board corners
            ret, corners = cv.findChessboardCorners(gray, CHECKERBOARD_SIZE, None)

            # If found, add object points, image points (after refining them)
            if ret == True:
                objpoints.append(objp)

                corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
                imgpoints.append(corners2)

                # Draw and display the corners
                cv.drawChessboardCorners(img, CHECKERBOARD_SIZE, corners2, ret)
                # cv.imshow("img", img)
                # cv.waitKey(500)

                cv.imwrite(str(CONTENT_DIR / "output" / (fname.name + "_corners.png")), img)

                # Calibration Part
                ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)  # type: ignore

                img = cv.imread(str(fname))

                if img is None:
                    continue

                h, w = img.shape[:2]
                # newcameramtx, roi = cv.getOptimalNewCameraMatrix(
                #     mtx, dist, (w, h), 0, (w, h)
                # )

                # dst = cv.undistort(img, mtx, dist, None, newcameramtx)

                # # crop the image
                # x, y, w, h = roi
                # dst = dst[y:y+h, x:x+w]
#
                # cv.imwrite(str(CONTENT_DIR / "output" / (fname.name + "_undistort.png")), dst)

                # Save results into calibration.json
                calibration_data[str(fname)] = (
                    {
                        "K": mtx.tolist(),  # Intrinsic matrix
                        "D": dist.tolist(),  # Distortion coefficients
                        "R": [
                            cv.Rodrigues(r)[0].tolist() for r in rvecs
                        ],  # Convert rvec to rotation matrix
                        "t": [t.tolist() for t in tvecs],  # Translation vectors
                        "width": w,
                        "height": h,
                    }
                )

                mean_error = 0
                for i in range(len(objpoints)):
                    imgpoints2, _ = cv.projectPoints(
                        objpoints[i], rvecs[i], tvecs[i], mtx, dist
                    )
                    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2) / len(imgpoints2)
                    mean_error += error

                print("total error: {}".format(mean_error / len(objpoints)))

        with open("calibration.json", "w") as f:
            json.dump(calibration_data, f, indent=2)

        return "Calibration successful"

### Visualization

### Utils

In [123]:
class Utils:
    @staticmethod
    def project_points(W: np.ndarray,
                       Lambda: np.ndarray,
                       Rt: np.ndarray) -> np.ndarray:
        """ Helper function to project 3D points to 2D image plane"""

        # Convert points to homogeneous coordinates
        W_tilde = np.vstack((W, np.ones((1, W.shape[1]))))

        print(f"W_tilde = \n{W_tilde}\n")

        # Calculate perspective projection in homogeneous coordinates
        X_tilde = Lambda @ Rt @ W_tilde

        print(f"X_tilde =  \n{X_tilde}\n")

        # Apply perspective division to convert coordinates from homogeneous to Cartesian
        X_tilde /= X_tilde[2,:]
        # Convert coordinates from homogeneous to Cartesian
        X = X_tilde[0:2,:]

        return X

    @staticmethod
    def draw_coordinate_frame(image_points, img):
        x0, y0 = image_points[:,0].astype(int)
        cv.circle(img, (x0, y0), 9, (0, 0, 0), -1)

        x1, y1 = image_points[:,1].astype(int)
        img = cv.arrowedLine(img, (x0, y0), (x1, y1), (255, 0, 0), 5)

        x2, y2 = image_points[:,2].astype(int)
        img = cv.arrowedLine(img, (x0, y0), (x2, y2), (0, 255, 0), 5)

        x3, y3 = image_points[:,3].astype(int)
        img = cv.arrowedLine(img, (x0, y0), (x3, y3), (0, 0, 255), 5)

        plt.imshow(img)

    @staticmethod
    def build_Lambda(phi_x, phi_y, skew, delta_x, delta_y):
        """ Build the intrinsic camera matrix Lambda """
        Lambda = np.array([[phi_x,  skew, delta_x],
                           [    0, phi_y, delta_y],
                           [    0,     0,       1]])
        return Lambda

    @staticmethod
    def json_read(filename):
        # Parses the json file
        try:
            with open(os.path.abspath(filename)) as f:
                data = json.load(f)
            return data
        except:
            raise ValueError("Unable to read JSON {}".format(filename))

### Main

In [124]:
class Visualization:
    @staticmethod
    def calibration_matrices(view: dict) -> tuple[np.ndarray, ...]:
        # Get the calibration matrices
        Lambda = np.array(view["K"])  # Intrinsic params.
        Omega = np.array(view["R"])  # Rotation
        tau = np.array(view["t"])  # Translation
        dist = np.array(view["D"])  # Lens distortion

        Lambda = Lambda.reshape(3, 3)
        Omega = Omega.reshape(3, 3)
        tau = tau.reshape(3, 1)

        return (Lambda, Omega, tau, dist)

    @staticmethod
    def undistort(view: dict, img: cv.typing.MatLike):
        Lambda, Omega, tau, dist = Visualization.calibration_matrices(view)

        fig = plt.figure()
        plt.imshow(img)

        scale_factor = 2
        W = scale_factor * np.array(
            [[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], dtype=np.float64
        )

        rvec = cv.Rodrigues(Omega)[0]
        tvec = tau.reshape(3, 1)

        print(f"W = \n{W}\n")

        image_axes, jac = cv.projectPoints(W, rvec, tvec, Lambda, dist)
        image_axes = image_axes.squeeze().T
        print(f"Projected image points = \n{image_axes}\n")

        Utils.draw_coordinate_frame(image_axes, img)

        return fig

    @staticmethod
    def plot_camera(
        Lambda: np.ndarray, Omega: np.ndarray, tau: np.ndarray, dimensions: list[int]
    ):
        sensor_size = np.array(dimensions)
        intrinsic_matrix = Lambda

        virtual_image_distance = 0.1

        # This is the camera coordinate frame
        # Camera pose, i.e., the matrix [R t] of extrinsic parameters
        Rt = np.block([Omega.T, -Omega.T @ tau])

        # Convert Rt from 3x4 to a 4x4 transformation matrix
        Rt = np.vstack([Rt, [0, 0, 0, 1]])

        # Print camera extrinsic parameters

        cam2world = Rt
        ax = pt.plot_transform(
            A2B=cam2world,
            s=2,
            # name="Camera"
        )

        pc.plot_camera(
            ax,
            cam2world=cam2world,
            M=intrinsic_matrix,
            sensor_size=sensor_size,
            virtual_image_distance=virtual_image_distance,
        )


        return ax

    @staticmethod
    def get_calibration_data():
        with open("calibration.json", "r") as f:
            calibration_data: dict[str, dict] = json.loads(f.read())

        return calibration_data

    @staticmethod
    def plot_cameras():
        calibration_data = Visualization.get_calibration_data()

        fig = plt.figure()
        ax = None

        for img in images:
            view = calibration_data[str(img)]

            img = cv.imread(str(images[0]))

            if img is None:
                continue

            Lambda, Omega, tau, _ = Visualization.calibration_matrices(view)

            ax = Visualization.plot_camera(
                Lambda,
                Omega,
                tau,
                img.shape[:2]
            )


        cam2world = pt.transform_from_pq([0, 0, 0, 0, 0, 0, 0])
        pt.plot_transform(
            ax,
            A2B=cam2world,
            s=3,
            # name="World"
        )

        if ax is not None:
          ax.view_init(30, 70)
          ax.set_xlim(-32, 32)
          ax.set_ylim(-32, 32)
          ax.set_zlim(-32, 32)

        return fig


    @staticmethod
    def show_world_axis():
        calibration_data = Visualization.get_calibration_data()

        bufs = []
        for i, (img) in enumerate(images):
            print(img)
            if i >= 5:
                break

            view = calibration_data[str(img)]

            img = cv.imread(str(CONTENT_DIR / "output" / (img.name + "_corners.png")))

            if img is None:
                continue

            bufs.append(Visualization.undistort(view, img))

        return bufs

    @staticmethod
    def distort_preview():
        normal = plt.figure()

        img = cv.imread(str(images[0]))

        if img is None:
            raise Exception("Failed to read img")

        plt.imshow(img)

        undistorted = plt.figure()

        calibration_data = Visualization.get_calibration_data()

        Lambda, _, _, dist = Visualization.calibration_matrices(calibration_data[str(images[0])])
        img_corrected = cv.undistort(img, Lambda, dist)
        plt.imshow(img_corrected)

        return [normal, undistorted]

    @staticmethod
    def run():
        camera = Visualization.plot_cameras()
        axises = Visualization.show_world_axis()
        distortion = Visualization.distort_preview()

        return [camera, *axises, *distortion]

### Gradio UI

In [125]:


class GradioUI:
  @staticmethod
  def save_images(images):
    for image in images:
      destination = image_dir / Path(image.name).name
      with open(image.name, "rb") as src, open(destination, "wb") as dst:
        dst.write(src.read())

  @staticmethod
  def run():
    with gradio.Blocks() as ui:
      gradio.Markdown("### Calibration UI")
      upload = gradio.Files(file_types=[".jpeg", ".jpg"])
      upload_button = gradio.Button("Upload Images")
      upload_output = gradio.Textbox(label="Upload status")

      calibrate_button = gradio.Button("Run Calibration")

      visualize_button = gradio.Button("Visualize Results")

      upload_button.click(fn=GradioUI.save_images, inputs=upload, outputs=upload_output)
      calibrate_button.click(fn=Calibration.run)
      visualize_button.click(fn=Visualization.run, outputs=[gradio.Plot() for _ in range(8)])

      ui.launch(share=False, inline=True)

## Interact

In [126]:
GradioUI.run()

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

/content/images/IMG_6729.jpg
total error: 0.02678475785334184
/content/images/IMG_6707.jpg
total error: 0.02519672355084055
/content/images/IMG_6708.jpg
total error: 0.02479310666729245
/content/images/IMG_6703.jpg
total error: 0.025691361905869604
/content/images/IMG_6715.jpg
total error: 0.02559616109420546
/content/images/IMG_6724.jpg
total error: 0.023948555113978816
/content/images/IMG_6722.jpg
total error: 0.02652852587837522
/content/images/IMG_6731.jpg
total error: 0.027756331510946114
/content/images/IMG_6717.jpg
total error: 0.029422015166252154
/content/images/IMG_6728.jpg
total error: 0.02228891050019366
/content/images/IMG_6721.jpg
total error: 0.028620303766020056
/content/images/IMG_6733.jpg
total error: 0.02679203189000108
/content/images/IMG_6705.jpg
total error: 0.026549450994940303
/content/images/IMG_6713.jpg
total error: 0.03169090484093545
/content/images/IMG_6711.jpg
total error: 0.02993629320094708
/content/images/IMG_6718.jpg
total error: 0.03316297275714069
/c

/usr/local/lib/python3.12/dist-packages/gradio/blocks.py:1974: UserWarning: A function (run) returned too many output values (needed: 0, returned: 1). Ignoring extra values.
    Output components:
        []
    Output values returned:
        ["Calibration successful"]
  warnings.warn(


/content/images/IMG_6729.jpg
W = 
[[0. 2. 0. 0.]
 [0. 0. 2. 0.]
 [0. 0. 0. 2.]]

Projected image points = 
[[2412.9344471  2485.81335023 2086.05009208 2275.25110555]
 [2110.19263386 2434.24203033 2103.82061341 2279.78223804]]

/content/images/IMG_6707.jpg
W = 
[[0. 2. 0. 0.]
 [0. 0. 2. 0.]
 [0. 0. 0. 2.]]

Projected image points = 
[[2365.73489656 2350.11976237 1965.34226934 2382.19447618]
 [2158.88503174 2552.46002523 2150.11627609 2265.43851192]]

/content/images/IMG_6708.jpg
W = 
[[0. 2. 0. 0.]
 [0. 0. 2. 0.]
 [0. 0. 0. 2.]]

Projected image points = 
[[2361.59079022 2358.37075562 1957.24083221 2329.38747281]
 [2152.41993656 2547.9367901  2147.02605492 2286.5417634 ]]

/content/images/IMG_6703.jpg
W = 
[[0. 2. 0. 0.]
 [0. 0. 2. 0.]
 [0. 0. 0. 2.]]

Projected image points = 
[[2374.3037208  2338.24447949 1995.74715801 2439.43920754]
 [2198.07683978 2581.61744301 2191.63873001 2318.92652112]]

/content/images/IMG_6715.jpg
W = 
[[0. 2. 0. 0.]
 [0. 0. 2. 0.]
 [0. 0. 0. 2.]]

Projected i

KeyboardInterrupt: 